In [53]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn

import numpy as np
import torch.optim as optim
import torch.utils.data as data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import ast
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [7]:
dataset = pd.read_csv('../../assets/labeled_dataset.csv')
def is_float(num):
    try:
        float(num)
        return True
    except ValueError:
        return False
    
dataset['landmarks'] = dataset['landmarks'].apply(lambda arr: np.array([float(n) for n in arr.split() if is_float(n)]))

### Hypothesis:
	- We can calculate the average of each landmark for a sequence of data
	- And have the average joint position for each joint in a sequence of 5 frames
	- We can train a regular NN to do classification or also regression because we would also have a good or bad ratio

In [13]:
dataset.head(12)

,video,group,frame,landmarks,Label
0,video56.mp4,1,5,"[0.0, 0.0, 51.864, 255.39, 103.26, 469.14, 0.9...",bad
1,video56.mp4,1,11,"[0.0, 0.0, 82.174, 253.09, 164.85, 471.52, 0.9...",bad
2,video56.mp4,1,17,"[0.0, 0.0, 118.78, 253.62, 238.0, 472.77, 0.93...",bad
3,video56.mp4,1,23,"[0.0, 0.0, 155.33, 253.25, 309.87, 471.26, 0.9...",bad
4,video56.mp4,1,29,"[0.0, 0.0, 181.27, 252.78, 362.54, 470.1, 0.94...",bad
5,video56.mp4,1,35,"[0.0, 0.0, 195.85, 253.87, 332.73, 475.31, 0.9...",bad
6,video56.mp4,1,41,"[0.0, 0.0, 232.89, 279.67, 260.39, 426.85, 0.9...",bad
7,video56.mp4,1,47,"[0.0, 0.0, 278.54, 286.11, 245.45, 414.3, 0.94...",bad
8,video56.mp4,1,53,"[0.0, 0.0, 291.42, 293.58, 253.71, 398.67, 0.9...",bad
9,video56.mp4,1,59,"[0.0, 0.0, 295.83, 291.14, 241.17, 403.39, 0.9...",bad


In [74]:
# grouing data
grouped = dataset.groupby(['video', 'group'])
static_transformed = {'video': [], 'group': [], 'landmark': [], 'label': []}

for (video, group), group_data in grouped:
    # Categorizing labels to get percentage
    sequence_labels = group_data['Label'].values
    binary_labels = np.array([1 if label == 'good' else 0 for label in sequence_labels])
        
    # Get a static landmark set
    # TODO: Enhance this static landmark (fill empty spaces in some cases + other processing)
    sequence_landmarks = group_data['landmarks'].values
    average_landmarks = np.mean(sequence_landmarks, axis=0)
    probability_good = np.mean(binary_labels)
    
    static_transformed['video'] += [video]
    static_transformed['group'] += [group]
    static_transformed['landmark'] += [average_landmarks]
    static_transformed['label'] += [probability_good]


In [50]:
static = pd.DataFrame(data=static_transformed)

In [51]:
static

,video,group,landmark,label
0,video17.mp4,1,"[0.0, 0.0, 272.42099999999994, 257.643, 371.19...",0.1
1,video17.mp4,2,"[0.0, 0.0, 272.42099999999994, 257.643, 371.19...",0.0
2,video17.mp4,3,"[0.0, 0.0, 272.42099999999994, 257.643, 371.19...",0.0
3,video17.mp4,4,"[0.0, 0.0, 272.42099999999994, 257.643, 371.19...",0.0
4,video17.mp4,5,"[0.0, 0.0, 272.42099999999994, 257.643, 371.19...",0.0
...,...,...,...,...
414,video56.mp4,4,"[0.0, 0.0, 272.42099999999994, 257.643, 371.19...",0.5
415,video56.mp4,5,"[0.0, 0.0, 272.42099999999994, 257.643, 371.19...",0.0
416,video56.mp4,6,"[0.0, 0.0, 272.42099999999994, 257.643, 371.19...",0.0
417,video56.mp4,7,"[0.0, 0.0, 272.42099999999994, 257.643, 371.19...",0.0


In [52]:
X = np.array(static['landmark'].tolist())
y = np.array(static['label'])

In [55]:
# splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# normalizing the landmarks
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# to tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

In [59]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_prob):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [71]:
input_size = X_train.shape[1]
hidden_size = 32
dropout_prob = 0.25  # Adjust dropout probability

net = Net(input_size, hidden_size, dropout_prob)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [72]:

num_epochs = 50
for epoch in range(num_epochs):
    # optimizer.zero_grad()
    outputs = net(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.view(-1, 1))
    loss.backward()
    optimizer.step()
    
    # converting to binary just to get the accuracy
    binary_y_train = (y_train >= 0.5).astype(int)
    
    with torch.no_grad():
        train_outputs = net(X_train_tensor)
        train_predicted_labels = (train_outputs >= 0.5).squeeze().cpu().numpy()
        train_accuracy = accuracy_score(binary_y_train, train_predicted_labels)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.4f}")



Epoch [1/50], Loss: 0.3202, Train Accuracy: 0.7672
Epoch [2/50], Loss: 0.3167, Train Accuracy: 0.7672
Epoch [3/50], Loss: 0.3170, Train Accuracy: 0.7672
Epoch [4/50], Loss: 0.3176, Train Accuracy: 0.7672
Epoch [5/50], Loss: 0.3164, Train Accuracy: 0.7672
Epoch [6/50], Loss: 0.3168, Train Accuracy: 0.7672
Epoch [7/50], Loss: 0.3143, Train Accuracy: 0.7672
Epoch [8/50], Loss: 0.3156, Train Accuracy: 0.7672
Epoch [9/50], Loss: 0.3169, Train Accuracy: 0.7672
Epoch [10/50], Loss: 0.3153, Train Accuracy: 0.7672
Epoch [11/50], Loss: 0.3138, Train Accuracy: 0.7672
Epoch [12/50], Loss: 0.3149, Train Accuracy: 0.7672
Epoch [13/50], Loss: 0.3138, Train Accuracy: 0.7672
Epoch [14/50], Loss: 0.3141, Train Accuracy: 0.7672
Epoch [15/50], Loss: 0.3157, Train Accuracy: 0.7672
Epoch [16/50], Loss: 0.3130, Train Accuracy: 0.7672
Epoch [17/50], Loss: 0.3124, Train Accuracy: 0.7672
Epoch [18/50], Loss: 0.3140, Train Accuracy: 0.7672
Epoch [19/50], Loss: 0.3132, Train Accuracy: 0.7672
Epoch [20/50], Loss: 

In [65]:
with torch.no_grad():
    net.eval()
    test_outputs = net(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor.view(-1, 1))
    print(f"Test Loss: {test_loss.item():.4f}")

    # Convert predictions to binary labels
    predicted_labels = (test_outputs >= 0.5).squeeze().cpu().numpy()
    
    # Convert the continuous labels to binary labels
    binary_y_test = (y_test >= 0.5).astype(int)

    # Calculate accuracy and F1-score
    accuracy = accuracy_score(binary_y_test, predicted_labels)
    f1 = f1_score(binary_y_test, predicted_labels)
    print(f"Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}")


Test Loss: 0.1101
Accuracy: 0.8452, F1-score: 0.0000
